In [1]:
!pip install -q transformers accelerate bitsandbytes peft datasets trl sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 29.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 93.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 34.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 62.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 76.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 96.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/1

In [3]:
import os
import json
import random
import traceback
from typing import Dict, List, Tuple
import numpy as np
import torch

# LLM / embeddings libs (ensure installed)
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer

# chromadb
import chromadb
from chromadb.config import Settings

# -----------------------
# Initialize models (LoRA + Mistral)
# -----------------------
from peft import PeftModel

device = "cuda" if torch.cuda.is_available() else "cpu"

MODEL_NAME = "mistralai/Mistral-7B-v0.1"
LORA_DIR   = "/kaggle/input/generator/other/default/1/generator_lora_25k"
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
print(f"Loading tokenizer + base model + LoRA from {LORA_DIR} ...")

print(f"Loading tokenizer + base model + LoRA from {LORA_DIR} ...")

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # Load base model
    base_model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto",
    )

    # Attach LoRA
    model = PeftModel.from_pretrained(
        base_model,
        LORA_DIR,
        use_bfloat16=False,
        use_bnb=False
    )

    # 🔥 FORCE GPU USAGE
    model.to(device)

    model.eval()
    print("✓ LoRA model loaded successfully on GPU!")

except Exception as e:
    print(f"❌ Failed to load LoRA model: {e}")
    tokenizer = None
    model = None


# -----------------------
# Load Embeddings Model
# -----------------------
print(f"Loading embedding model: {EMBEDDING_MODEL_NAME} ...")
try:
    embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME)
except Exception as e:
    print(f"Warning: failed to load embedding model {EMBEDDING_MODEL_NAME}: {e}")
    embedding_model = None


2025-11-17 22:51:22.997276: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763419883.019492     176 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763419883.026298     176 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# -----------------------
# Chroma vector DB (persisted)
# -----------------------
from chromadb import PersistentClient

chroma_client = PersistentClient(path="./.chroma")
collection_name = "rag_chunks"
# Create collection if not exists, else get it
try:
    if collection_name in [c.name for c in chroma_client.list_collections()]:
        collection = chroma_client.get_collection(name=collection_name)
    else:
        collection = chroma_client.create_collection(name=collection_name, metadata={"hnsw:space": "cosine"})
except Exception as e:
    # fallback: create
    print(f"Warning creating/getting Chroma collection: {e}")
    collection = chroma_client.create_collection(name=collection_name, metadata={"hnsw:space": "cosine"})

# -----------------------
# Helper functions
# -----------------------
def generate_text(prompt: str, max_new_tokens: int = 512, temperature: float = 0.7) -> str:
    """Generate text using the loaded model/tokenizer. If model/tokenizer missing, returns a placeholder."""
    if model is None or tokenizer is None:
        return "MODEL_NOT_LOADED"
    formatted_prompt = f"[INST] {prompt} [/INST]"
    inputs = tokenizer(formatted_prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.95,
            top_k=50,
            pad_token_id=tokenizer.eos_token_id,
        )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "[/INST]" in generated_text:
        generated_text = generated_text.split("[/INST]")[-1].strip()
    return generated_text

def generate_embeddings_batch(chunks_batch: List[str]) -> List[List[float]]:
    """Return list of embeddings (lists). If embedding_model missing, return zeros."""
    if embedding_model is None:
        # fallback: zeros
        dim = 384
        return [[0.0] * dim for _ in chunks_batch]
    embeddings = embedding_model.encode(chunks_batch, convert_to_numpy=True)
    # embeddings shape (n, dim)
    return embeddings.tolist()

# -----------------------
# Data loading / preprocessing
# -----------------------
def load_glaive_dataset(file_path: str, num_samples: int = 1000) -> List[Dict[str, str]]:
    print(f"Loading Glaive dataset from {file_path}...")
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    random.shuffle(data)
    data = data[:num_samples]
    print(f"Loaded {len(data)} samples from Glaive dataset")
    return data

def create_document_chunks_from_glaive(data: List[Dict[str, str]]) -> List[str]:
    chunks = []
    for idx, item in enumerate(data, 1):
        question = item.get('question', '')
        answer = item.get('answer', '')
        chunk = f"<id:{idx}>\n{question}\n{answer}\n</id:{idx}>"
        chunks.append(chunk)
    print(f"Created {len(chunks)} document chunks for RAG retrieval")
    return chunks

def preprocess_text(text: str) -> str:
    return text.lower()

def preprocess_chunks(chunks: List[str]) -> List[str]:
    return [preprocess_text(chunk) for chunk in chunks]

# -----------------------
# Embeddings + vector store
# -----------------------
def generate_embeddings(chunks: List[str], batch_size: int = 32, verbose: bool = False) -> np.ndarray:
    all_embeddings = []
    if verbose:
        print(f"Generating embeddings for {len(chunks)} chunks...")
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]
        embeddings = generate_embeddings_batch(batch)
        all_embeddings.extend(embeddings)
        if verbose and ((i + batch_size) % 100 == 0 or i + batch_size >= len(chunks)):
            print(f"Processed {min(i + batch_size, len(chunks))}/{len(chunks)} chunks")
    return np.array(all_embeddings)

def save_embeddings(embeddings: np.ndarray, output_file: str) -> None:
    try:
        with open(output_file, 'w', encoding='utf-8') as file:
            json.dump(embeddings.tolist(), file)
        print(f"Saved embeddings to {output_file}")
    except Exception as e:
        print(f"Warning: failed to save embeddings to {output_file}: {e}")

def add_to_vector_store(embeddings: np.ndarray, chunks: List[str]):
    ids = [f"chunk_{i}" for i in range(len(chunks))]
    try:
        collection.add(
            ids=ids,
            embeddings=embeddings.tolist(),
            documents=chunks
        )
        print(f"Added {len(chunks)} chunks to Chroma collection '{collection_name}'")
    except Exception as e:
        print(f"Error adding to vector store: {e}")
        traceback.print_exc()

def similarity_search(query_embedding: np.ndarray, top_k: int = 5) -> List[str]:
    try:
        results = collection.query(
            query_embeddings=[query_embedding.tolist()],
            n_results=top_k
        )
        # results is a dict; documents is a list of lists
        return results.get("documents", [[]])[0]
    except Exception as e:
        print(f"Warning during similarity_search: {e}")
        return []

def retrieve_relevant_chunks(query_text: str, top_k: int = 5) -> List[str]:
    query_embedding = generate_embeddings([query_text])[0]
    relevant_chunks = similarity_search(query_embedding, top_k=top_k)
    return relevant_chunks

# -----------------------
# Reward functions
# -----------------------
def calculate_rouge_score(response: str, ground_truth: str) -> float:
    def lcs_length(s1, s2):
        m, n = len(s1), len(s2)
        dp = [[0] * (n + 1) for _ in range(m + 1)]
        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if s1[i-1] == s2[j-1]:
                    dp[i][j] = dp[i-1][j-1] + 1
                else:
                    dp[i][j] = max(dp[i-1][j], dp[i][j-1])
        return dp[m][n]
    resp_tokens = response.lower().split()
    gt_tokens = ground_truth.lower().split()
    if len(resp_tokens) == 0 or len(gt_tokens) == 0:
        return 0.0
    lcs_len = lcs_length(resp_tokens, gt_tokens)
    precision = lcs_len / len(resp_tokens) if len(resp_tokens) > 0 else 0
    recall = lcs_len / len(gt_tokens) if len(gt_tokens) > 0 else 0
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)

def calculate_token_overlap(response: str, ground_truth: str) -> float:
    resp_tokens = set(response.lower().split())
    gt_tokens = set(ground_truth.lower().split())
    if len(gt_tokens) == 0:
        return 0.0
    intersection = resp_tokens & gt_tokens
    return len(intersection) / len(gt_tokens)

def detect_hallucination(response: str, context_chunks: List[str]) -> float:
    response_lower = response.lower()
    context_combined = " ".join(context_chunks).lower()
    resp_terms = [w for w in response_lower.split() if len(w) > 3]
    if len(resp_terms) == 0:
        return 0.0
    found_terms = sum(1 for term in resp_terms if term in context_combined)
    coverage = found_terms / len(resp_terms)
    return coverage

def calculate_context_relevance(query: str, context_chunks: List[str]) -> float:
    if not context_chunks:
        return 0.0
    query_embedding = generate_embeddings([query])[0]
    context_embeddings = [generate_embeddings([chunk])[0] for chunk in context_chunks]
    similarities = [np.dot(query_embedding, ctx_emb) / (np.linalg.norm(query_embedding) * np.linalg.norm(ctx_emb) + 1e-12) for ctx_emb in context_embeddings]
    return float(np.mean(similarities)) if similarities else 0.0

def cosine_similarity(vec1: np.ndarray, vec2: np.ndarray) -> float:
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0.0
    return float(dot_product / (norm_vec1 * norm_vec2))

def calculate_hybrid_reward(
    response: str,
    ground_truth: str,
    query: str,
    context_chunks: List[str],
    weights: Dict[str, float] = None
) -> Dict[str, float]:
    if weights is None:
        weights = {
            'semantic_similarity': 0.35,
            'rouge_score': 0.25,
            'token_overlap': 0.15,
            'hallucination_penalty': 0.15,
            'context_relevance': 0.10
        }
    response_embedding = generate_embeddings([response])[0]
    ground_truth_embedding = generate_embeddings([ground_truth])[0]
    semantic_sim = cosine_similarity(response_embedding, ground_truth_embedding)
    rouge_score = calculate_rouge_score(response, ground_truth)
    token_overlap = calculate_token_overlap(response, ground_truth)
    hallucination_score = detect_hallucination(response, context_chunks)
    context_relevance = calculate_context_relevance(query, context_chunks)
    hybrid_reward = (
        weights['semantic_similarity'] * semantic_sim +
        weights['rouge_score'] * rouge_score +
        weights['token_overlap'] * token_overlap +
        weights['hallucination_penalty'] * hallucination_score +
        weights['context_relevance'] * context_relevance
    )
    return {
        'total_reward': hybrid_reward,
        'semantic_similarity': semantic_sim,
        'rouge_score': rouge_score,
        'token_overlap': token_overlap,
        'hallucination_score': hallucination_score,
        'context_relevance': context_relevance
    }

# -----------------------
# LLM response helpers
# -----------------------
def construct_prompt(query: str, context_chunks: List[str]) -> str:
    context = "\n".join(context_chunks)
    prompt = f"""You are a helpful coding assistant. Use the provided context to answer the question. If the context doesn't contain the information needed, say 'I don't have enough information to answer this question.'

Context:
{context}

Question: {query}

Answer:"""
    return prompt

def generate_response(prompt: str, max_tokens: int = 512, temperature: float = 0.7) -> str:
    return generate_text(prompt, max_new_tokens=max_tokens, temperature=temperature)

def basic_rag_pipeline(query: str) -> str:
    relevant_chunks = retrieve_relevant_chunks(query)
    prompt = construct_prompt(query, relevant_chunks)
    response = generate_response(prompt)
    return response

# -----------------------
# RL components: state/actions
# -----------------------
def define_state(
    query: str,
    context_chunks: List[str],
    rewritten_query: str = None,
    previous_responses: List[str] = None,
    previous_rewards: List[float] = None
) -> dict:
    return {
        "original_query": query,
        "current_query": rewritten_query if rewritten_query else query,
        "context": context_chunks,
        "previous_responses": previous_responses if previous_responses else [],
        "previous_rewards": previous_rewards if previous_rewards else []
    }

def define_action_space() -> List[str]:
    return ["rewrite_query", "expand_context", "filter_context", "generate_response"]

# -----------------------
# Load / initialize Q_VALUES (persisted)
# -----------------------
Q_FILE = "q_values.json"
try:
    if os.path.exists(Q_FILE):
        with open(Q_FILE, "r") as f:
            loaded = json.load(f)
        Q_VALUES = {a: float(loaded.get(a, 0.0)) for a in define_action_space()}
        print(f"Loaded saved Q-values from {Q_FILE}")
    else:
        Q_VALUES = {a: 0.0 for a in define_action_space()}
except Exception as e:
    print(f"Warning: could not load {Q_FILE}: {e}")
    Q_VALUES = {a: 0.0 for a in define_action_space()}

# -----------------------
# Query rewrite / context ops
# -----------------------
def rewrite_query(query: str, context_chunks: List[str], max_tokens: int = 100, temperature: float = 0.3) -> str:
    rewrite_prompt = f"""You are a query optimization assistant. Rewrite the given query to make it more effective for retrieving relevant information.

Original query: {query}

Context so far:
{' '.join(context_chunks[:2]) if context_chunks else 'No context available yet'}

Provide ONLY the rewritten query, nothing else. Make it more specific and targeted for better document retrieval.

Rewritten query:"""
    rewritten_query = generate_text(rewrite_prompt, max_new_tokens=max_tokens, temperature=temperature)
    return rewritten_query.strip()

def expand_context(query: str, current_chunks: List[str], top_k: int = 3) -> List[str]:
    additional_chunks = retrieve_relevant_chunks(query, top_k=top_k + len(current_chunks))
    new_chunks = [chunk for chunk in additional_chunks if chunk not in current_chunks]
    expanded_context = current_chunks + new_chunks[:top_k]
    return expanded_context

def filter_context(query: str, context_chunks: List[str]) -> List[str]:
    if not context_chunks:
        return []
    query_embedding = generate_embeddings([query])[0]
    chunk_embeddings = [generate_embeddings([chunk])[0] for chunk in context_chunks]
    relevance_scores = [cosine_similarity(query_embedding, chunk_embedding) for chunk_embedding in chunk_embeddings]
    sorted_chunks = [x for _, x in sorted(zip(relevance_scores, context_chunks), reverse=True)]
    filtered_chunks = sorted_chunks[:min(5, len(sorted_chunks))]
    return filtered_chunks

# -----------------------
# Q-learning policy
# -----------------------
def policy_network_q(epsilon: float = 0.1, train_mode: bool = True) -> str:
    action_space = define_action_space()
    if train_mode and np.random.random() < epsilon:
        return random.choice(action_space)
    return max(Q_VALUES, key=Q_VALUES.get)

def update_q_value(action: str, reward: float, lr: float = 0.1):
    old_value = Q_VALUES.get(action, 0.0)
    Q_VALUES[action] = old_value + lr * (reward - old_value)

# -----------------------
# RL step using Q-learning selection
# -----------------------
def rl_step(state: dict, ground_truth: str, query: str, train_mode: bool = True) -> tuple:
    action = policy_network_q(epsilon=0.1, train_mode=train_mode)
    response = None
    reward_info = None

    # ---------- INTERMEDIATE ACTIONS ----------
    if action == "rewrite_query":
        old_query = state["current_query"]
        rewritten_query = rewrite_query(old_query, state["context"])
        if rewritten_query:
            state["current_query"] = rewritten_query
            state["context"] = retrieve_relevant_chunks(rewritten_query)

        # Small reward: Did rewriting improve context?
        reward = calculate_context_relevance(query, state["context"])
        if train_mode:
            update_q_value(action, reward)
        reward_info = {"total_reward": reward}

    elif action == "expand_context":
        old_len = len(state["context"])
        state["context"] = expand_context(state["current_query"], state["context"])

        # Small reward: Did context grow?
        reward = len(state["context"]) - old_len
        reward = max(0, reward * 0.05)  # tiny reward
        if train_mode:
            update_q_value(action, reward)
        reward_info = {"total_reward": reward}

    elif action == "filter_context":
        old_context = state["context"]
        state["context"] = filter_context(state["current_query"], state["context"])

        # Small reward: Did it become more relevant?
        reward = calculate_context_relevance(query, state["context"])
        if train_mode:
            update_q_value(action, reward)
        reward_info = {"total_reward": reward}

    # ---------- FINAL ACTION ----------
    elif action == "generate_response":
        prompt = construct_prompt(state["current_query"], state["context"])
        response = generate_response(prompt)
        reward_info = calculate_hybrid_reward(response, ground_truth, query, state["context"])
        state["previous_responses"].append(response)
        state["previous_rewards"].append(reward_info['total_reward'])

        if train_mode:
            update_q_value(action, reward_info['total_reward'])

    return state, action, reward_info, response


# -----------------------
# Training loop
# -----------------------
def training_loop(
    training_samples: List[Dict[str, str]],
    num_episodes: int = 100,
    samples_per_episode: int = 100
) -> Tuple[List[float], List[Dict], List[str]]:
    all_rewards = []
    all_metrics = []
    best_responses = []

    print(f"\nStarting RL training for {num_episodes} episodes...")
    print(f"Using {samples_per_episode} random samples per episode\n")

    for episode in range(num_episodes):
        episode_samples = random.sample(training_samples, min(samples_per_episode, len(training_samples)))
        episode_rewards = []
        episode_metrics = []

        for sample_idx, sample in enumerate(episode_samples):
            # progress line
            print(f"Episode {episode+1}/{num_episodes} | Progress: {sample_idx+1}/{len(episode_samples)} samples", end="\r", flush=True)

            query = sample['question']
            ground_truth = sample['answer']
            context_chunks = retrieve_relevant_chunks(query)
            state = define_state(query, context_chunks)

            reward_info = None
            response = None
            for step in range(5):
                state, action, reward_info, response = rl_step(state, ground_truth, query, train_mode=True)
                if response and reward_info:
                    episode_rewards.append(reward_info['total_reward'])
                    episode_metrics.append(reward_info)
                    break

            # if RL didn't produce a response in the steps, force a generate_response
            if reward_info is None:
                prompt = construct_prompt(state["current_query"], state["context"])
                response = generate_response(prompt)
                reward_info = calculate_hybrid_reward(response, ground_truth, query, state["context"])
                episode_rewards.append(reward_info['total_reward'])
                episode_metrics.append(reward_info)
                update_q_value("generate_response", reward_info['total_reward'])

        avg_episode_reward = np.mean(episode_rewards) if episode_rewards else 0.0
        all_rewards.append(avg_episode_reward)

        print(f"\nEpisode {episode+1}/{num_episodes}: Avg Reward = {avg_episode_reward:.4f}")

    # --- persist average rewards and Q-values for plotting / re-use ---
    try:
        with open("training_avg_rewards.json", "w") as f:
            json.dump(all_rewards, f)
        print("Saved per-episode average rewards to training_avg_rewards.json")
    except Exception as e:
        print(f"Warning: could not save training_avg_rewards.json: {e}")

    try:
        with open("q_values.json", "w") as f:
            json.dump(Q_VALUES, f)
        print("Saved Q-values to q_values.json")
    except Exception as e:
        print(f"Warning: could not save q_values.json: {e}")

    print(f"\n{'='*60}")
    print(f"Training completed!")
    print(f"Final Average Reward: {np.mean(all_rewards[-10:]) if all_rewards else 0.0:.4f}")
    print(f"{'='*60}")
    return all_rewards, all_metrics, best_responses

In [6]:
# -----------------------
# Comparison / evaluation
# -----------------------
def compare_rag_on_multiple_samples(
    test_samples: List[Dict[str, str]],
    training_samples: List[Dict[str, str]],
    num_test_samples: int = 10
) -> Dict:
    print("\n" + "=" * 80)
    print(f"COMPARING RAG APPROACHES ON {num_test_samples} TEST SAMPLES")
    print("=" * 80)

    print("\n[1/2] Training RL-enhanced RAG...")
    rl_rewards, rl_metrics, _ = training_loop(training_samples, num_episodes=5, samples_per_episode=100)

    print(f"\n[2/2] Testing on {num_test_samples} samples...")
    simple_rag_results = []
    rl_rag_results = []

    for idx, sample in enumerate(test_samples[:num_test_samples]):
        query = sample['question']
        ground_truth = sample['answer']

        print(f"\n--- Test Sample {idx + 1}/{num_test_samples} ---")
        print(f"Query: {query[:100]}...")

        # Simple RAG baseline
        simple_response = basic_rag_pipeline(query)
        simple_context = retrieve_relevant_chunks(query)
        simple_metrics = calculate_hybrid_reward(simple_response, ground_truth, query, simple_context)
        simple_rag_results.append(simple_metrics)

        # RL-enhanced RAG (greedy policy during test)
        context_chunks = retrieve_relevant_chunks(query)
        state = define_state(query, context_chunks)

        reward_info = None
        response = None
        for step in range(5):
            state, action, reward_info, response = rl_step(state, ground_truth, query, train_mode=False)
            if response and reward_info:
                rl_rag_results.append(reward_info)
                break

        if reward_info is None:
            prompt = construct_prompt(state["current_query"], state["context"])
            response = generate_response(prompt)
            reward_info = calculate_hybrid_reward(response, ground_truth, query, state["context"])
            rl_rag_results.append(reward_info)

        rl_reward = reward_info['total_reward'] if reward_info else 0.0
        print(f"Simple RAG Reward: {simple_metrics['total_reward']:.4f}")
        print(f"RL-RAG Reward: {rl_reward:.4f}")

    simple_avg = np.mean([r['total_reward'] for r in simple_rag_results]) if simple_rag_results else 0.0
    rl_avg = np.mean([r['total_reward'] for r in rl_rag_results]) if rl_rag_results else 0.0
    improvement = ((rl_avg - simple_avg) / simple_avg * 100) if simple_avg > 0 else 0

    results = {
        'simple_rag': {'avg_reward': float(simple_avg), 'detailed_results': simple_rag_results},
        'rl_rag': {'avg_reward': float(rl_avg), 'detailed_results': rl_rag_results, 'training_rewards': rl_rewards},
        'improvement_percentage': float(improvement),
        'num_test_samples': num_test_samples
    }

    print("\n" + "=" * 80)
    print("FINAL EVALUATION RESULTS")
    print("=" * 80)
    print(f"Simple RAG Average Reward:      {simple_avg:.4f}")
    print(f"RL-enhanced RAG Average Reward: {rl_avg:.4f}")
    print(f"Improvement:                    {improvement:.2f}%")
    print("=" * 80)

    return results

In [ ]:
# -----------------------
# Main execution
# -----------------------
if __name__ == "__main__":
    print("\n" + "=" * 80)
    print("RAG with RL using Glaive Code Assistant Dataset")
    print("=" * 80)
    print(f"\nModel: {MODEL_NAME}")
    print(f"Embedding Model: {EMBEDDING_MODEL_NAME}")
    print(f"Device: {device}")
    print("=" * 80 + "\n")

    glaive_file_path = "/kaggle/input/glaive-coding-assistant/c9bc9129-eba0-4b10-8292-4ae70fc7fa0d.json"

    try:
        glaive_data = load_glaive_dataset(glaive_file_path, num_samples=1000)
        training_data = glaive_data[:900]
        test_data = glaive_data[900:]
        document_chunks = create_document_chunks_from_glaive(glaive_data)
        preprocessed_chunks = preprocess_chunks(document_chunks)

        print(f"📚 Loaded {len(glaive_data)} total samples")
        print(f"🏋️ Training samples: {len(training_data)}")
        print(f"🧪 Test samples: {len(test_data)}")
        print(f"📄 Document chunks for RAG: {len(document_chunks)}\n")

        # save the loaded 1000 Q/A to a text file for later inspection
        try:
            with open("glaive_1000_qas.txt", "w", encoding="utf-8") as out:
                for i, item in enumerate(glaive_data, 1):
                    q = item.get("question", "").strip()
                    a = item.get("answer", "").strip()
                    out.write(f"### Sample {i}\nQuestion:\n{q}\n\nAnswer:\n{a}\n\n\n")
            print("✓ Saved 1000 Q/A to glaive_1000_qas.txt")
        except Exception as e:
            print(f"Warning: could not save glaive_1000_qas.txt: {e}")

        print("Generating embeddings for document chunks...")
        embeddings = generate_embeddings(preprocessed_chunks, verbose=True)
        save_embeddings(embeddings, "embeddings.json")
        add_to_vector_store(embeddings, preprocessed_chunks)

        

        print("✓ Vector store ready!\n")

        results = compare_rag_on_multiple_samples(test_samples=test_data, training_samples=training_data, num_test_samples=10)

        # save results
        try:
            with open('rl_rag_multi_sample_results.json', 'w') as f:
                json.dump(results, f, indent=2, default=str)
            print("\n✓ Results saved to 'rl_rag_multi_sample_results.json'")
        except Exception as e:
            print(f"Warning: could not save results file: {e}")

    except FileNotFoundError:
        print(f"❌ Error: Could not find Glaive dataset at {glaive_file_path}")
    except Exception as e:
        print(f"❌ Error occurred: {str(e)}")
        traceback.print_exc()


RAG with RL using Glaive Code Assistant Dataset

Model: mistralai/Mistral-7B-v0.1
Embedding Model: sentence-transformers/all-MiniLM-L6-v2
Device: cuda

Loading Glaive dataset from /kaggle/input/glaive-coding-assistant/c9bc9129-eba0-4b10-8292-4ae70fc7fa0d.json...
Loaded 1000 samples from Glaive dataset
Created 1000 document chunks for RAG retrieval
📚 Loaded 1000 total samples
🏋️ Training samples: 900
🧪 Test samples: 100
📄 Document chunks for RAG: 1000

✓ Saved 1000 Q/A to glaive_1000_qas.txt
Generating embeddings for document chunks...
Generating embeddings for 1000 chunks...
Processed 800/1000 chunks
Processed 1000/1000 chunks
Saved embeddings to embeddings.json
Added 1000 chunks to Chroma collection 'rag_chunks'
✓ Vector store ready!


COMPARING RAG APPROACHES ON 10 TEST SAMPLES

[1/2] Training RL-enhanced RAG...

Starting RL training for 5 episodes...
Using 100 random samples per episode

Episode 1/5 | Progress: 100/100 samples
Episode 1/5: Avg Reward = 0.2696


In [ ]:
import json
import matplotlib.pyplot as plt

# Load rewards from JSON
json_path = "/kaggle/working/training_avg_rewards.json"
with open(json_path, "r") as f:
    avg_rewards = json.load(f)

# Create x-axis values (episodes)
episodes = list(range(1, len(avg_rewards) + 1))

# Plot
plt.figure(figsize=(12, 6))
plt.plot(episodes, avg_rewards, marker='o', linewidth=2)
plt.title("Average Reward per Episode", fontsize=20)
plt.xlabel("Episode", fontsize=14)
plt.ylabel("Average Reward", fontsize=14)
plt.grid(True, linestyle='--', alpha=0.6)

plt.xticks(episodes)  # Show all episode numbers
plt.show()


In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# -------------------------------------------------------------------------
# LOAD JSON (from file OR fallback to provided JSON content)
# -------------------------------------------------------------------------

json_path = Path("/kaggle/working/rl_rag_multi_sample_results.json")

if json_path.exists():
    data = json.loads(json_path.read_text())
else:
    # fallback – paste the JSON content you gave earlier
    data = json.loads(r'''
    PASTE YOUR FULL JSON HERE IF NEEDED
    ''')

# -------------------------------------------------------------------------
# EXTRACT DATA
# -------------------------------------------------------------------------

simple = data["simple_rag"]
rl = data["rl_rag"]

def extract_metric(results, key):
    return np.array([r[key] for r in results])

simple_total = extract_metric(simple["detailed_results"], "total_reward")
rl_total = extract_metric(rl["detailed_results"], "total_reward")

metrics = [
    "semantic_similarity",
    "rouge_score",
    "token_overlap",
    "hallucination_score",
    "context_relevance"
]

simple_metrics = {m: extract_metric(simple["detailed_results"], m) for m in metrics}
rl_metrics = {m: extract_metric(rl["detailed_results"], m) for m in metrics}

samples = np.arange(1, len(simple_total) + 1)

# -------------------------------------------------------------------------
# 1) ZOOMED TOTAL REWARD COMPARISON
# -------------------------------------------------------------------------

plt.figure(figsize=(8,4.2))
plt.plot(samples, simple_total, marker='o', label="Simple RAG")
plt.plot(samples, rl_total, marker='o', label="RL-RAG")

ymin = min(simple_total.min(), rl_total.min()) - 0.02
ymax = max(simple_total.max(), rl_total.max()) + 0.02
plt.ylim(ymin, ymax)

plt.title("Per-sample Total Reward Comparison (Zoomed)")
plt.xlabel("Sample Index")
plt.ylabel("Total Reward")
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.tight_layout()
plt.show()

# -------------------------------------------------------------------------
# 2) DIFFERENCE PLOT (RL – Simple)
# -------------------------------------------------------------------------

plt.figure(figsize=(8,3.6))
diff_total = rl_total - simple_total
plt.plot(samples, diff_total, marker='o', color='purple')

plt.axhline(0, linestyle='--', color='gray')
plt.title("Difference (RL - Simple) in Total Reward")
plt.xlabel("Sample Index")
plt.ylabel("Difference")
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# -------------------------------------------------------------------------
# 3) AVERAGE METRIC COMPARISON (GROUPED BAR)
# -------------------------------------------------------------------------

avg_simple = [simple_metrics[m].mean() for m in metrics]
avg_rl = [rl_metrics[m].mean() for m in metrics]

x = np.arange(len(metrics))
width = 0.35

plt.figure(figsize=(9,4.2))
plt.bar(x - width/2, avg_simple, width, label='Simple RAG')
plt.bar(x + width/2, avg_rl, width, label='RL-RAG')

plt.xticks(x, metrics, rotation=20)
plt.title("Average Metric Comparison (Lower is Better for Hallucination Score)")
plt.ylabel("Average Value")
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

# -------------------------------------------------------------------------
# 4) METRIC DIFFERENCE HEATMAP
# -------------------------------------------------------------------------

diff_matrix = np.vstack([rl_metrics[m] - simple_metrics[m] for m in metrics])

plt.figure(figsize=(9,3.8))
img = plt.imshow(diff_matrix, aspect='auto', interpolation='nearest', cmap='viridis')
plt.colorbar(img, label="RL - Simple")

plt.xticks(np.arange(len(samples)), samples)
plt.yticks(np.arange(len(metrics)), metrics)

plt.title("Per-sample Metric Differences (Positive = RL Higher)")
plt.xlabel("Sample Index")
plt.tight_layout()
plt.show()

# -------------------------------------------------------------------------
# 5) TRAINING REWARD HISTORY (IF AVAILABLE)
# -------------------------------------------------------------------------

if "training_rewards" in rl:
    tr = np.array(rl["training_rewards"])

    plt.figure(figsize=(8,3.4))
    plt.plot(np.arange(1, len(tr)+1), tr, marker='o', color='orange')
    plt.title("RL-RAG Training Reward History")
    plt.xlabel("Episode")
    plt.ylabel("Avg Reward")
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()


In [3]:
import os
import json
import random
import traceback
from typing import Dict, List, Tuple
import numpy as np
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
from chromadb import PersistentClient
from peft import PeftModel

# -----------------------
# Configuration
# -----------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "mistralai/Mistral-7B-v0.1"
LORA_DIR = "/kaggle/input/generator/other/default/1/generator_lora_25k"
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
Q_FILE = "/kaggle/input/qvalue/q_values (2).json"
CHROMA_PATH = "./.chroma"
COLLECTION_NAME = "rag_chunks"

# -----------------------
# Load Models
# -----------------------
print(f"Loading models on {device}...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    base_model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto",
    )
    model = PeftModel.from_pretrained(base_model, LORA_DIR, use_bfloat16=False, use_bnb=False)
    model.to(device)
    model.eval()
    print("✓ LoRA model loaded successfully!")
except Exception as e:
    print(f"❌ Failed to load model: {e}")
    model = None
    tokenizer = None

try:
    embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME)
    print("✓ Embedding model loaded!")
except Exception as e:
    print(f"❌ Failed to load embedding model: {e}")
    embedding_model = None

# -----------------------
# Load/Create Chroma Vector DB
# -----------------------
try:
    chroma_client = PersistentClient(path=CHROMA_PATH)
    # Check if collection exists
    if COLLECTION_NAME in [c.name for c in chroma_client.list_collections()]:
        collection = chroma_client.get_collection(name=COLLECTION_NAME)
        print(f"✓ Connected to existing Chroma collection '{COLLECTION_NAME}'")
    else:
        collection = chroma_client.create_collection(name=COLLECTION_NAME, metadata={"hnsw:space": "cosine"})
        print(f"✓ Created new Chroma collection '{COLLECTION_NAME}'")
except Exception as e:
    print(f"❌ Failed to initialize Chroma: {e}")
    collection = None

# -----------------------
# Load Pre-trained Q-values
# -----------------------
try:
    if os.path.exists(Q_FILE):
        with open(Q_FILE, "r") as f:
            Q_VALUES = json.load(f)
        print(f"✓ Loaded Q-values from {Q_FILE}")
        print(f"   Q-values: {Q_VALUES}")
    else:
        print(f"⚠️  Q-values file '{Q_FILE}' not found! Using default values.")
        Q_VALUES = {"rewrite_query": 0.0, "expand_context": 0.0, "filter_context": 0.0, "generate_response": 0.0}
except Exception as e:
    print(f"❌ Failed to load Q-values: {e}")
    Q_VALUES = {"rewrite_query": 0.0, "expand_context": 0.0, "filter_context": 0.0, "generate_response": 0.0}

# -----------------------
# Helper Functions
# -----------------------
def generate_text(prompt: str, max_new_tokens: int = 512, temperature: float = 0.7) -> str:
    """Generate text using the loaded model."""
    if model is None or tokenizer is None:
        return "MODEL_NOT_LOADED"
    formatted_prompt = f"[INST] {prompt} [/INST]"
    inputs = tokenizer(formatted_prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.95,
            top_k=50,
            pad_token_id=tokenizer.eos_token_id,
        )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "[/INST]" in generated_text:
        generated_text = generated_text.split("[/INST]")[-1].strip()
    return generated_text

def generate_embeddings_batch(chunks_batch: List[str]) -> List[List[float]]:
    """Generate embeddings for a batch of text chunks."""
    if embedding_model is None:
        dim = 384
        return [[0.0] * dim for _ in chunks_batch]
    embeddings = embedding_model.encode(chunks_batch, convert_to_numpy=True)
    return embeddings.tolist()

def generate_embeddings(chunks: List[str], batch_size: int = 32, verbose: bool = False) -> np.ndarray:
    """Generate embeddings for all chunks."""
    all_embeddings = []
    if verbose:
        print(f"Generating embeddings for {len(chunks)} chunks...")
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]
        embeddings = generate_embeddings_batch(batch)
        all_embeddings.extend(embeddings)
        if verbose and ((i + batch_size) % 100 == 0 or i + batch_size >= len(chunks)):
            print(f"Processed {min(i + batch_size, len(chunks))}/{len(chunks)} chunks")
    return np.array(all_embeddings)

def retrieve_relevant_chunks(query_text: str, top_k: int = 5) -> List[str]:
    """Retrieve relevant chunks from vector store."""
    if collection is None:
        return []
    try:
        query_embedding = generate_embeddings([query_text])[0]
        results = collection.query(
            query_embeddings=[query_embedding.tolist()],
            n_results=top_k
        )
        return results.get("documents", [[]])[0]
    except Exception as e:
        print(f"❌ Error during retrieval: {e}")
        return []

def construct_prompt(query: str, context_chunks: List[str]) -> str:
    context = "\n".join(context_chunks[:2])
    prompt = f"""You are a helpful coding assistant. Use the provided context to answer the question. If the context doesn't contain the information needed, say 'I don't have enough information to answer this question.'
--------------------------
Context:
{context}
---------------------------
Question: {query}

Answer:"""
    return prompt

def generate_response(prompt: str, max_tokens: int = 1000, temperature: float = 0.7) -> str:
    return generate_text(prompt, max_new_tokens=max_tokens, temperature=temperature)

def cosine_similarity(vec1: np.ndarray, vec2: np.ndarray) -> float:
    """Calculate cosine similarity between two vectors."""
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0.0
    return float(dot_product / (norm_vec1 * norm_vec2))

def calculate_rouge_score(response: str, ground_truth: str) -> float:
    """Calculate ROUGE-L score."""
    def lcs_length(s1, s2):
        m, n = len(s1), len(s2)
        dp = [[0] * (n + 1) for _ in range(m + 1)]
        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if s1[i-1] == s2[j-1]:
                    dp[i][j] = dp[i-1][j-1] + 1
                else:
                    dp[i][j] = max(dp[i-1][j], dp[i][j-1])
        return dp[m][n]
    
    resp_tokens = response.lower().split()
    gt_tokens = ground_truth.lower().split()
    if len(resp_tokens) == 0 or len(gt_tokens) == 0:
        return 0.0
    lcs_len = lcs_length(resp_tokens, gt_tokens)
    precision = lcs_len / len(resp_tokens) if len(resp_tokens) > 0 else 0
    recall = lcs_len / len(gt_tokens) if len(gt_tokens) > 0 else 0
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)

def calculate_token_overlap(response: str, ground_truth: str) -> float:
    """Calculate token overlap between response and ground truth."""
    resp_tokens = set(response.lower().split())
    gt_tokens = set(ground_truth.lower().split())
    if len(gt_tokens) == 0:
        return 0.0
    intersection = resp_tokens & gt_tokens
    return len(intersection) / len(gt_tokens)

def detect_hallucination(response: str, context_chunks: List[str]) -> float:
    """Detect hallucination by checking if response terms are in context."""
    response_lower = response.lower()
    context_combined = " ".join(context_chunks).lower()
    resp_terms = [w for w in response_lower.split() if len(w) > 3]
    if len(resp_terms) == 0:
        return 0.0
    found_terms = sum(1 for term in resp_terms if term in context_combined)
    coverage = found_terms / len(resp_terms)
    return coverage

def calculate_context_relevance(query: str, context_chunks: List[str]) -> float:
    """Calculate relevance of context to query."""
    if not context_chunks:
        return 0.0
    query_embedding = generate_embeddings([query])[0]
    context_embeddings = [generate_embeddings([chunk])[0] for chunk in context_chunks]
    similarities = [
        np.dot(query_embedding, ctx_emb) / (np.linalg.norm(query_embedding) * np.linalg.norm(ctx_emb) + 1e-12)
        for ctx_emb in context_embeddings
    ]
    return float(np.mean(similarities)) if similarities else 0.0

def calculate_hybrid_reward(
    response: str,
    ground_truth: str,
    query: str,
    context_chunks: List[str],
    weights: Dict[str, float] = None
) -> Dict[str, float]:
    """Calculate comprehensive reward metrics."""
    if weights is None:
        weights = {
            'semantic_similarity': 0.35,
            'rouge_score': 0.25,
            'token_overlap': 0.15,
            'hallucination_penalty': 0.15,
            'context_relevance': 0.10
        }
    
    response_embedding = generate_embeddings([response])[0]
    ground_truth_embedding = generate_embeddings([ground_truth])[0]
    semantic_sim = cosine_similarity(response_embedding, ground_truth_embedding)
    rouge_score = calculate_rouge_score(response, ground_truth)
    token_overlap = calculate_token_overlap(response, ground_truth)
    hallucination_score = detect_hallucination(response, context_chunks)
    context_relevance = calculate_context_relevance(query, context_chunks)
    
    hybrid_reward = (
        weights['semantic_similarity'] * semantic_sim +
        weights['rouge_score'] * rouge_score +
        weights['token_overlap'] * token_overlap +
        weights['hallucination_penalty'] * hallucination_score +
        weights['context_relevance'] * context_relevance
    )
    
    return {
        'total_reward': hybrid_reward,
        'semantic_similarity': semantic_sim,
        'rouge_score': rouge_score,
        'token_overlap': token_overlap,
        'hallucination_score': hallucination_score,
        'context_relevance': context_relevance
    }

# -----------------------
# Data Loading
# -----------------------
def load_glaive_dataset(file_path: str, num_samples: int = 1000) -> List[Dict[str, str]]:
    """Load Glaive dataset from JSON file."""
    print(f"Loading Glaive dataset from {file_path}...")
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    random.shuffle(data)
    data = data[:num_samples]
    print(f"Loaded {len(data)} samples from Glaive dataset")
    return data

def create_document_chunks_from_glaive(data: List[Dict[str, str]]) -> List[str]:
    """Create document chunks from Glaive data."""
    chunks = []
    for idx, item in enumerate(data, 1):
        question = item.get('question', '')
        answer = item.get('answer', '')
        chunk = f"<id:{idx}>\n{question}\n{answer}\n</id:{idx}>"
        chunks.append(chunk)
    print(f"Created {len(chunks)} document chunks for RAG retrieval")
    return chunks

def preprocess_text(text: str) -> str:
    """Preprocess text to lowercase."""
    return text.lower()

def preprocess_chunks(chunks: List[str]) -> List[str]:
    """Preprocess all chunks."""
    return [preprocess_text(chunk) for chunk in chunks]

def add_to_vector_store(embeddings: np.ndarray, chunks: List[str]):
    """Add embeddings and chunks to Chroma vector store."""
    if collection is None:
        print("❌ Collection not initialized, cannot add to vector store")
        return
    ids = [f"chunk_{i}" for i in range(len(chunks))]
    try:
        collection.add(
            ids=ids,
            embeddings=embeddings.tolist(),
            documents=chunks
        )
        print(f"✓ Added {len(chunks)} chunks to Chroma collection '{COLLECTION_NAME}'")
    except Exception as e:
        print(f"❌ Error adding to vector store: {e}")
        traceback.print_exc()

# -----------------------
# Basic RAG Pipeline
# -----------------------
def basic_rag_pipeline(query: str) -> tuple:
    """Execute basic RAG without RL."""
    relevant_chunks = retrieve_relevant_chunks(query)
    prompt = construct_prompt(query, relevant_chunks)
    response = generate_response(prompt)
    return response, relevant_chunks

# -----------------------
# RL-Enhanced RAG Actions
# -----------------------
def rewrite_query(query: str, context_chunks: List[str]) -> str:
    """Rewrite query for better retrieval."""
    rewrite_prompt = f"""You are a query optimization assistant. Rewrite the given query to make it more effective for retrieving relevant information.

Original query: {query}

Context so far:
{' '.join(context_chunks[:2]) if context_chunks else 'No context available yet'}

Provide ONLY the rewritten query, nothing else. Make it more specific and targeted for better document retrieval.

Rewritten query:"""
    rewritten_query = generate_text(rewrite_prompt, max_new_tokens=100, temperature=0.3)
    return rewritten_query.strip()

def expand_context(query: str, current_chunks: List[str], top_k: int = 3) -> List[str]:
    """Expand context with additional chunks."""
    additional_chunks = retrieve_relevant_chunks(query, top_k=top_k + len(current_chunks))
    new_chunks = [chunk for chunk in additional_chunks if chunk not in current_chunks]
    expanded_context = current_chunks + new_chunks[:top_k]
    return expanded_context

def filter_context(query: str, context_chunks: List[str]) -> List[str]:
    """Filter context to keep only most relevant chunks."""
    if not context_chunks:
        return []
    query_embedding = generate_embeddings([query])[0]
    chunk_embeddings = [generate_embeddings([chunk])[0] for chunk in context_chunks]
    relevance_scores = [cosine_similarity(query_embedding, chunk_embedding) for chunk_embedding in chunk_embeddings]
    sorted_chunks = [x for _, x in sorted(zip(relevance_scores, context_chunks), reverse=True)]
    filtered_chunks = sorted_chunks[:min(5, len(sorted_chunks))]
    return filtered_chunks

def select_best_action() -> str:
    """Select action with highest Q-value (greedy policy for inference)."""
    return max(Q_VALUES, key=Q_VALUES.get)

def rl_enhanced_rag_pipeline(query: str, max_steps: int = 5) -> tuple:
    """Execute RL-enhanced RAG using pre-trained Q-values."""
    # Initialize state
    context_chunks = retrieve_relevant_chunks(query)
    current_query = query
    actions_taken = []
    
    for step in range(max_steps):
        action = select_best_action()
        actions_taken.append(action)
        
        if action == "rewrite_query":
            current_query = rewrite_query(current_query, context_chunks)
            context_chunks = retrieve_relevant_chunks(current_query)
            
        elif action == "expand_context":
            context_chunks = expand_context(current_query, context_chunks)
            
        elif action == "filter_context":
            context_chunks = filter_context(current_query, context_chunks)
            
        elif action == "generate_response":
            prompt = construct_prompt(current_query, context_chunks)
            response = generate_response(prompt)
            return response, context_chunks, actions_taken
    
    # If we exit loop without generating, generate response
    prompt = construct_prompt(current_query, context_chunks)
    response = generate_response(prompt)
    return response, context_chunks, actions_taken

# -----------------------
# Inference Function
# -----------------------
def run_inference(query: str, ground_truth: str = None):
    """Run inference comparing normal RAG vs RL-enhanced RAG."""
    print("\n" + "=" * 80)
    print("RAG INFERENCE - COMPARISON")
    print("=" * 80)
    print(f"\nQuery: {query}\n")
    
    # Normal RAG
    print("🔵 Running Normal RAG...")
    normal_response, normal_context = basic_rag_pipeline(query)
    print(f"\nNormal RAG Response:\n{normal_response}\n")
    
    # RL-Enhanced RAG
    print("🟢 Running RL-Enhanced RAG...")
    rl_response, rl_context, actions_taken = rl_enhanced_rag_pipeline(query)
    print(f"\nRL-Enhanced RAG Response:\n{rl_response}\n")
    print(f"Actions taken: {' → '.join(actions_taken)}\n")
    
    # Calculate rewards if ground truth provided
    if ground_truth:
        print("📊 Calculating Reward Scores...")
        normal_metrics = calculate_hybrid_reward(normal_response, ground_truth, query, normal_context)
        rl_metrics = calculate_hybrid_reward(rl_response, ground_truth, query, rl_context)
        
        print("\n" + "-" * 80)
        print("REWARD SCORES")
        print("-" * 80)
        print(f"\n🔵 Normal RAG Total Reward: {normal_metrics['total_reward']:.4f}")
        print(f"   - Semantic Similarity: {normal_metrics['semantic_similarity']:.4f}")
        print(f"   - ROUGE Score: {normal_metrics['rouge_score']:.4f}")
        print(f"   - Token Overlap: {normal_metrics['token_overlap']:.4f}")
        print(f"   - Hallucination Score: {normal_metrics['hallucination_score']:.4f}")
        print(f"   - Context Relevance: {normal_metrics['context_relevance']:.4f}")
        
        print(f"\n🟢 RL-Enhanced RAG Total Reward: {rl_metrics['total_reward']:.4f}")
        print(f"   - Semantic Similarity: {rl_metrics['semantic_similarity']:.4f}")
        print(f"   - ROUGE Score: {rl_metrics['rouge_score']:.4f}")
        print(f"   - Token Overlap: {rl_metrics['token_overlap']:.4f}")
        print(f"   - Hallucination Score: {rl_metrics['hallucination_score']:.4f}")
        print(f"   - Context Relevance: {rl_metrics['context_relevance']:.4f}")
        
        improvement = ((rl_metrics['total_reward'] - normal_metrics['total_reward']) / 
                      normal_metrics['total_reward'] * 100) if normal_metrics['total_reward'] > 0 else 0
        print(f"\n📈 Improvement: {improvement:+.2f}%")
        print("-" * 80)
        
        return {
            'query': query,
            'ground_truth': ground_truth,
            'normal_rag': {
                'response': normal_response,
                'context_chunks': normal_context,
                'metrics': normal_metrics
            },
            'rl_rag': {
                'response': rl_response,
                'context_chunks': rl_context,
                'actions_taken': actions_taken,
                'metrics': rl_metrics
            },
            'improvement_percentage': improvement
        }
    else:
        print("ℹ️  No ground truth provided - skipping reward calculation")
        return {
            'query': query,
            'normal_rag': {'response': normal_response, 'context_chunks': normal_context},
            'rl_rag': {'response': rl_response, 'context_chunks': rl_context, 'actions_taken': actions_taken}
        }



2025-11-18 04:08:38.293688: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763438918.535460      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763438918.595485      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✓ Embedding model loaded!
✓ Connected to existing Chroma collection 'rag_chunks'
✓ Loaded Q-values from /kaggle/input/qvalue/q_values (2).json
   Q-values: {'rewrite_query': 0.18677308480050683, 'expand_context': 0.051585000000000006, 'filter_context': 0.08103826938353614, 'generate_response': 0.5009191809725395}


In [4]:
# -----------------------
# Main Execution
# -----------------------

print("\n" + "=" * 80)
print("RAG WITH RL - SETUP AND INFERENCE")
print("=" * 80)

glaive_file_path = "/kaggle/input/glaive-coding-assistant/c9bc9129-eba0-4b10-8292-4ae70fc7fa0d.json"


# Load dataset and create vector store
print("\n[STEP 1] Loading Dataset and Creating Vector Store...")
glaive_data = load_glaive_dataset(glaive_file_path, num_samples=1000)

# Create chunks
document_chunks = create_document_chunks_from_glaive(glaive_data)
preprocessed_chunks = preprocess_chunks(document_chunks)

# Generate embeddings
print("\n[STEP 2] Generating Embeddings...")
embeddings = generate_embeddings(preprocessed_chunks, verbose=True)

# Add to vector store
print("\n[STEP 3] Adding to Vector Store...")
add_to_vector_store(embeddings, preprocessed_chunks)

print("\n✓ Vector store setup complete!\n")

# Save the Q&A data for reference
try:
    with open("glaive_1000_qas.txt", "w", encoding="utf-8") as out:
        for i, item in enumerate(glaive_data, 1):
            q = item.get("question", "").strip()
            a = item.get("answer", "").strip()
            out.write(f"### Sample {i}\nQuestion:\n{q}\n\nAnswer:\n{a}\n\n\n")
    print("✓ Saved 1000 Q/A to glaive_1000_qas.txt\n")
except Exception as e:
    print(f"⚠️  Warning: could not save glaive_1000_qas.txt: {e}\n")




RAG WITH RL - SETUP AND INFERENCE

[STEP 1] Loading Dataset and Creating Vector Store...
Loading Glaive dataset from /kaggle/input/glaive-coding-assistant/c9bc9129-eba0-4b10-8292-4ae70fc7fa0d.json...
Loaded 1000 samples from Glaive dataset
Created 1000 document chunks for RAG retrieval

[STEP 2] Generating Embeddings...
Generating embeddings for 1000 chunks...
Processed 800/1000 chunks
Processed 1000/1000 chunks

[STEP 3] Adding to Vector Store...
✓ Added 1000 chunks to Chroma collection 'rag_chunks'

✓ Vector store setup complete!

✓ Saved 1000 Q/A to glaive_1000_qas.txt



In [7]:
# Now run inference examples
print("\n" + "=" * 80)
print("RUNNING INFERENCE EXAMPLES")
print("=" * 80)

# Example 1: With ground truth
query1 = "How do I create a list comprehension in Python?"
ground_truth1 = "List comprehensions in Python provide a concise way to create lists. The syntax is [expression for item in iterable if condition]. For example: squares = [x**2 for x in range(10)] creates a list of squares."

results1 = run_inference(query1)

# Example 2: From actual dataset
print("\n\n" + "=" * 80)
print("EXAMPLE 2: Using sample from dataset")
print("=" * 80)

sample = random.choice(glaive_data)
query2 = sample['question']
ground_truth2 = sample['answer']

results2 = run_inference(query2)

# Save results
try:
    all_results = {
        'example_1': results1,
        'example_2': results2
    }
    with open('inference_results.json', 'w') as f:
        json.dump(all_results, f, indent=2, default=str)
    print("\n✓ Results saved to 'inference_results.json'")
except Exception as e:
    print(f"\n⚠️  Failed to save results: {e}")

print("\n" + "=" * 80)
print("SETUP AND INFERENCE COMPLETE!")
print("=" * 80)
print("\nYou can now use run_inference() with your own queries:")
print("  run_inference('your query here', 'optional ground truth')")
print("=" * 80)
    



RUNNING INFERENCE EXAMPLES

RAG INFERENCE - COMPARISON

Query: How do I create a list comprehension in Python?

🔵 Running Normal RAG...

Normal RAG Response:
[INST] You are a helpful coding assistant. Use the provided context to answer the question. If the context doesn't contain the information needed, say 'I don't have enough information to answer this question.'

Context:
<id:560>
i have a table called "employees" and i want to find the age of the oldest employee. how can i write a sql query to achieve this?
to find the age of the oldest employee in the "employees" table, you can use the following sql query:

```sql
select max(age) from employees;
```

this query selects the maximum (oldest) value from the "age" column in the "employees" table. the max() function returns the largest value within a set, which in this case is the oldest age. by executing this query, you will obtain the age of the oldest employee in the table.
</id:560>
<id:580>
in python 3.4, the new module enum has 

KeyboardInterrupt: 

In [5]:
import gradio as gr
import json

def format_metrics(metrics):
    """Format metrics dictionary into readable string."""
    if not metrics:
        return "No metrics available"
    
    return f"""
**Total Reward:** {metrics.get('total_reward', 0):.4f}
- Semantic Similarity: {metrics.get('semantic_similarity', 0):.4f}
- ROUGE Score: {metrics.get('rouge_score', 0):.4f}
- Token Overlap: {metrics.get('token_overlap', 0):.4f}
- Hallucination Score: {metrics.get('hallucination_score', 0):.4f}
- Context Relevance: {metrics.get('context_relevance', 0):.4f}
"""

def format_context_chunks(chunks):
    """Format context chunks into readable string."""
    if not chunks:
        return "No context chunks available"
    
    formatted = ""
    for i, chunk in enumerate(chunks, 1):
        formatted += f"**Chunk {i}:**\n{chunk}\n\n"
    return formatted

def process_query(query, ground_truth=None):
    """Process query and return formatted results."""
    # Call your run_inference function
    results = run_inference(query, ground_truth)
    
    # Extract data
    normal_rag = results.get('normal_rag', {})
    rl_rag = results.get('rl_rag', {})
    
    # Format outputs
    normal_response = normal_rag.get('response', 'No response available')
    normal_context = format_context_chunks(normal_rag.get('context_chunks', []))
    normal_metrics = format_metrics(normal_rag.get('metrics'))
    
    rl_response = rl_rag.get('response', 'No response available')
    rl_context = format_context_chunks(rl_rag.get('context_chunks', []))
    rl_actions = ", ".join(rl_rag.get('actions_taken', [])) if rl_rag.get('actions_taken') else "No actions recorded"
    rl_metrics = format_metrics(rl_rag.get('metrics'))
    
    improvement = results.get('improvement_percentage', 0)
    improvement_text = f"**Improvement:** {improvement:+.2f}%" if ground_truth else "No ground truth provided"

    
    
    return (
        normal_response,
        normal_context,
        normal_metrics,
        rl_response,
        rl_context,
        rl_actions,
        rl_metrics,
        improvement_text
    )

# Create Gradio interface
with gr.Blocks(title="RAG Comparison Interface", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🔍 RAG Inference Comparison")
    gr.Markdown("Compare Normal RAG vs RL-Enhanced RAG performance")
    
    with gr.Row():
        with gr.Column():
            query_input = gr.Textbox(
                label="Query",
                placeholder="Enter your question here...",
                lines=3
            )
            ground_truth_input = gr.Textbox(
                label="Ground Truth (Optional)",
                placeholder="Enter expected answer for metrics calculation...",
                lines=3
            )
            submit_btn = gr.Button("Run Inference", variant="primary", size="lg")
    
    gr.Markdown("---")
    
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## 🔵 Normal RAG")
            
            gr.Markdown("### Response")
            normal_response_output = gr.Textbox(
                label="",
                lines=6,
                interactive=False,
                show_label=False
            )
            
            gr.Markdown("---")
            gr.Markdown("### Context Chunks")
            normal_context_output = gr.Markdown()
            
            gr.Markdown("---")
            gr.Markdown("### Metrics")
            normal_metrics_output = gr.Markdown()
        
        with gr.Column(scale=1):
            gr.Markdown("## 🟢 RL-Enhanced RAG")
            
            gr.Markdown("### Response")
            rl_response_output = gr.Textbox(
                label="",
                lines=6,
                interactive=False,
                show_label=False
            )
            
            gr.Markdown("---")
            gr.Markdown("### Context Chunks")
            rl_context_output = gr.Markdown()
            
            gr.Markdown("---")
            gr.Markdown("### Actions Taken")
            rl_actions_output = gr.Textbox(
                label="",
                lines=2,
                interactive=False,
                show_label=False
            )
            
            gr.Markdown("---")
            gr.Markdown("### Metrics")
            rl_metrics_output = gr.Markdown()
    
    gr.Markdown("---")
    
    with gr.Row():
        improvement_output = gr.Markdown(label="Improvement")
    
    # Connect the button to the processing function
    submit_btn.click(
        fn=process_query,
        inputs=[query_input, ground_truth_input],
        outputs=[
            normal_response_output,
            normal_context_output,
            normal_metrics_output,
            rl_response_output,
            rl_context_output,
            rl_actions_output,
            rl_metrics_output,
            improvement_output
        ]
    )
    
    # Example queries
    gr.Markdown("---")
    gr.Markdown("### 💡 Example Queries")
    examples = gr.Examples(
        examples=[
            ["What is machine learning?", "Machine learning is a subset of AI that enables systems to learn from data."],
            ["Explain reinforcement learning", ""],
            ["What are the benefits of RAG systems?", ""]
        ],
        inputs=[query_input, ground_truth_input]
    )

# Launch the interface
if __name__ == "__main__":
    demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://bfb7da42f18cb763e6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



RAG INFERENCE - COMPARISON

Query: How do I create a list comprehension in Python?

🔵 Running Normal RAG...

Normal RAG Response:
You can create a list comprehension in Python by using the following syntax:

```python
[expression for item in sequence]
```

In this syntax, 'expression' is a computation that is applied to each element in 'sequence', and the resulting value is added to the list. The 'item' variable represents the current element being iterated over in the sequence.

Here's an example of how you can use a list comprehension to create a list of squares:

```python
squares = [x**2 for x in range(1, 11)]
print(squares)  # Output: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
```

In this code, the list comprehension is defined as `[x**2 for x in range(1, 11)]`. The expression `x**2` is applied to each element in the sequence `range(1, 11)`, which results in a list of squares from 1 to 10.

List comprehensions can be a useful tool for creating lists in Python, especially when you ne

In [6]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()
torch.cuda.synchronize()
